In [201]:
import pandas as pd
import numpy as np
import aocd
import timeit
import os
from aocd.models import Puzzle
os.environ['AOC_SESSION'] = '53616c7465645f5fda624994231306c055524663b055568600cbd7a8e885d012db0c35ac4c420439e127563fce396857'

In [202]:
import timeit

# 3

In [138]:
puzzle = Puzzle(year=2019, day=3)

In [139]:
def define_instructions(inp_data):
    instructions = inp_data.split('\n')
    instructions = [instruction.split(',') for instruction in instructions]
    instructions = [[(move[0], int(move[1:])) for move in instruction] for instruction in instructions ]
    return instructions

In [140]:
instructions = define_instructions(puzzle.input_data)

In [141]:
test_input= """R8,U5,L5,D3
U7,R6,D4,L4"""
test_input2 = """R75,D30,R83,U83,L12,D49,R71,U7,L72
U62,R66,U55,R34,D71,R55,D58,R83"""
test_input3 = """R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
U98,R91,D20,R16,D67,R40,U7,R15,U6,R7"""

In [289]:
define_instructions(test_input)

[[('R', 8), ('U', 5), ('L', 5), ('D', 3)],
 [('U', 7), ('R', 6), ('D', 4), ('L', 4)]]

In [142]:
test_instructions = define_instructions(test_input)
test_instructions2 = define_instructions(test_input2)
test_instructions3 = define_instructions(test_input3)

In [224]:
# Deprecated: np matrix way of mapping
x_0 = 1000
y_0 = 1000
N_grid = 2 * x_0 

def matrix_mapping(wire_map, instruction):
    
    x = x_0
    y = y_0
    
    for direction, step_size in instruction:
        
        if direction is 'U':
            wire_map[x , y + 1 : y + step_size + 1] += 1
            y = y + step_size
        elif direction is 'D':
            wire_map[x , y - step_size : y ] += 1
            y = y - step_size
        elif direction is 'R':
            wire_map[x + 1 : x + step_size + 1 , y] += 1
            x = x + step_size
        elif direction is 'L':
            wire_map[x - step_size : x, y] += 1
            x = x - step_size
        
    return wire_map

def total_wire_mapping(instructions):
    
    wire_map = np.zeros((N_grid,N_grid))
    
    for instruction in instructions:
        wire_map = matrix_mapping(wire_map, instruction)

    return wire_map


def get_closest_manhattan(instructions):
    
    wire_map = total_wire_mapping(instructions)
    wire_map[x_0,y_0] = 0
    
    crossings = np.argwhere(wire_map > 1)
    #for x,y in crossings:
        #print(abs(x-x_0))
        #print(abs(y-y_0))
    closest = min([(abs(x-x_0) + abs(y-y_0)) for x,y in crossings])
    
    return closest

def get_closest_manhattan2(instructions):
    
    wiremap_1 = matrix_mapping(np.zeros((N_grid,N_grid)), instructions[0])
    wiremap_2 = matrix_mapping(np.zeros((N_grid,N_grid)), instructions[1])
    
    wiremap_total = wiremap_1 + wiremap_2
    
    wiremap_total[x_0,y_0] = 0
    
    crossings = np.argwhere(wiremap_total > 1)
    
    closest = min([(abs(x-x_0) + abs(y-y_0)) for x,y in crossings])
    
    return closest

In [321]:
# def wire_mapping_old(instruction):
#     x = 0
#     y = 0
    
#     wire_map = []
    
#     for direction, step_size in instruction:
#         if direction is 'U':
#             for step in np.arange(step_size):
#                 wire_map.append((x, y + step + 1))
#             y = y + step_size
#         elif direction is 'D':
#             for step in np.arange(step_size):
#                 wire_map.append((x, y - step - 1))
#             y = y - step_size
#         elif direction is 'R':
#             for step in np.arange(step_size):
#                 wire_map.append((x + step + 1, y))
#             x = x + step_size
#         elif direction is 'L':
#             for step in np.arange(step_size):
#                 wire_map.append((x - step - 1, y))
#             x = x - step_size
        
#     return wire_map

def step(x,y,direction):
    if direction is 'U':
        return x, y + 1
    elif direction is 'D':
        return x, y - 1
    elif direction is 'R':
        return x + 1, y
    elif direction is 'L':
        return x - 1, y

    
def wire_mapping(instruction):
    
    x = 0
    y = 0
    wire_map = [(0,0)]
    
    for direction, step_size in instruction:
        for i in np.arange(1,step_size+1):
            x,y = step(x,y,direction)
            wire_map.append((x,y))

    # remove self crossings
    wire_map = list(set(wire_map))
            
    return wire_map


def two_wire_mapping(instructions):
    
    wire_map = []
    
    for instruction in instructions:
        wire_map += wire_mapping(instruction)
    
    return wire_map

def get_closest_manhattan(instructions):
    
    wire_map = two_wire_mapping(instructions)
    
    #print(f"wire map of length {len(wire_map)}")
    
    crossings = [key for key, count in Counter(wire_map).items() if count > 1]
    
    print(crossings)
    
    crossings.remove((0,0))
    
    manhattan_distances = [(abs(x) + abs(y)) for x,y in crossings]
    
    if len(manhattan_distances):
        return min(manhattan_distances)
    
    else:
        return "No crossing found!"

def get_closest_manhattan_from_input(input_string):
    
    instructions = define_instructions(input_string)
    
    return get_closest_manhattan(instructions)

def get_crossings(instructions):
    
    wire_map = two_wire_mapping(instructions)
    
    #print(f"wire map of length {len(wire_map)}")
    
    crossings = [key for key, count in Counter(wire_map).items() if count > 1]

    return crossings
    
def steps_to_crossing(instruction, crossing):
    x=0
    y=0
    steps = 0
    for direction, step_size in instruction:
        for i in np.arange(1,step_size+1):
            x,y = step(x,y,direction)
            steps += 1
            #print((x,y))
            #print(crossing)
            if (x,y) == crossing:
                #print('crossing reached')
                return steps

def get_combined_steps(instructions):
    
    crossings = get_crossings(instructions)
    crossings.remove((0,0))
    #print(crossings)
    steps_to_crossings = []
    
    steps_to_crossings = [[steps_to_crossing(instruction, crossing) for instruction in instructions] for crossing in crossings]
      
    return min([steps1 + steps2 for steps1,steps2 in steps_to_crossings])    
    #return steps_to_crossings

In [297]:
start = timeit.default_timer()

print(get_closest_manhattan_from_input(test_input))

stop = timeit.default_timer()

print('Time: ', stop - start)  

start = timeit.default_timer()

print(get_closest_manhattan_from_input(test_input2))

stop = timeit.default_timer()

print('Time: ', stop - start)  

start = timeit.default_timer()

print(get_closest_manhattan_from_input(test_input3))

stop = timeit.default_timer()

print('Time: ', stop - start) 

start = timeit.default_timer()

print(get_closest_manhattan_from_input(puzzle.input_data))

stop = timeit.default_timer()

print('Time: ', stop - start) 

[(3, 3), (0, 0), (6, 5)]
6
Time:  0.0002712890000111656
[(155, 11), (0, 0), (155, 4), (146, 46), (158, -12)]
159
Time:  0.0008396049997827504
[(0, 0), (107, 47), (124, 11), (107, 51), (107, 71), (157, 18)]
135
Time:  0.0006973620002099779
[(970, 1387), (2546, 1383), (3385, 1259), (2326, 1383), (0, 0), (4503, 1752), (1403, 1850), (970, 1159), (2511, 1383), (4052, 1974), (3791, 1598), (4303, 1752), (1906, 1428), (3791, 1974), (2072, 1383), (1381, 1428), (1381, 1565), (2487, 1383)]
2129
Time:  0.31303882999964117


In [322]:
get_combined_steps(define_instructions(test_input))

30

In [304]:
(3,3) is (3,3)

True

In [316]:
steps_to_crossing(test_instructions[0],(3,3))

crossing reached


20

In [325]:
get_combined_steps(instructions)

134662